In [2]:
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time

nft_collection_df = pd.read_csv('eth_nft_30days.csv') #get sigdev nft rankings for the 30days

In [220]:
# cleaning the csv. file to the right format
for index, row in nft_collection_df.iterrows():
    mystring =row['COLLECTION'].split(">",1)[1]
    mystring = mystring.split("<")[0]
    mystring = mystring.replace("'", " ")
    mystring = mystring.replace("[", " ")
    mystring = mystring.replace("]", " ")
    mystring = mystring.replace(":", " ")
    mystring = mystring.replace("  ", " ")
    mystring = mystring.replace(" - ", " ")

    nft_collection_df.at[index, 'collection'] = mystring

In [222]:
# filtering out some nft collections
nft_collection_df['collection'] = np.where((nft_collection_df.collection == 'Sandbox s LANDs'),'Sandbox Land',nft_collection_df.collection)
nft_collection_df['collection'] = np.where((nft_collection_df.collection == 'Primera by Mitchell and Yun'),'Primera',nft_collection_df.collection)

#not enough data for 0'clocks nft
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == '7 O clock'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == '3 O clock'].index, inplace = True)
#sql error -  bind parameter
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Dci'].index, inplace = True) 
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'AK Ultra'].index, inplace = True) 
# twitter api data = 0 error {'meta': {'result_count': 0}}
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'VOX Series 3'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Beeple Spring Collection'].index, inplace = True)
nft_collection_df.drop(nft_collection_df[nft_collection_df.collection == 'Bored & Dangerous'].index, inplace = True)

In [10]:
"""
trial run for the first two groups - and it works!
"""
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time
import os

engine = db.create_engine("")
connection = engine.connect()

bearer_token = ''

nft_dict = dict(zip(nft_collection_df.collection, nft_collection_df.index))
top_100_collections = list(nft_dict.keys())
nested_top_100_collections = [top_100_collections[n:n+10] for n in range(0, len(top_100_collections), 10)]
nested_top_100_collections

for i in range(len(nested_top_100_collections)): # paginating to account for twitter api rate limit 
    for item in nested_top_100_collections[i]:
        print(item)
        headers = {'Authorization': f"Bearer {bearer_token}",}
        params = {
            'query': item + ' "nft" -is:retweet',
            "tweet.fields": "created_at,public_metrics,text",
            "sort_order": "relevancy",
            "max_results": 100,
            "expansions": "author_id",
        }
        response = requests.get('https://api.twitter.com/2/tweets/search/recent', params=params, headers=headers)
        data = response.json()
        data = pd.DataFrame(data['data'])
        data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)
        result = data[['created_at', 'author_id', 'text','retweet_count', 'reply_count', 'like_count', 'id']]
        result = result.replace('\n',' ', regex=True)
        result = result.replace("'"," ", regex=True)
        result = result.replace("'s", " s", regex=True)
        
        s = ', '.join(["("+ ', '.join( ["'" + item + "'", "'" + str(result.iloc[i].created_at)[0:19]+ "'",
              "'"+str(result.iloc[i].author_id)+"'", "'"+str(result.iloc[i].text)+"'", "'" + str(result.iloc[i].retweet_count)+ "'", "'"+str(result.iloc[i].reply_count)+"'", "'"+str(result.iloc[i].like_count)+"'", "'"+str(result.iloc[i].id)+"'"+ ")" ]) for i in range(len(result))])
        
        query = f"""
        INSERT INTO nft_tweets_demo 
        VALUES {s}
        ON CONFLICT ON CONSTRAINT unique_tweet_id 
        DO NOTHING
        """
        try:
            pd.read_sql_query(db.text(query), connection)

        except Exception as e:
            print(e)
            print(item)

    print('---loop---')

LILVERSE
Invisible Friends
WINGS GENESIS
ALPACADABRAZ
Something
CloneX
Yakuza Pandas
Great Expectations
PudgyPenguins
MultibeastsByHaas
8PES
goblintown
NetvrkLand
SHIBOSHIS
Azimuth Points
Jenkins the Valet
Art Blocks
Impostors Genesis
Cool Cats
GoblinVerse
STOICS
omgkirby x CT
Spooktaculars
Cold Blooded Creepz
SAN Origin
ENS
DeadFellaz
Hashmasks
parallel
Emblem Vault V2
Neo Tokyo Citizen
Unidentified contract
Elves Forest NFT
BoredApeYachtClub
Lazy Ape Yacht Club
Frank Stella Geometries
World Of Women
Akutars
Dooplicator
FewoWorld Paint
Gala Games
CryptoDickbutts S3
HsGenesisPass
The Plague
y00ts Yacht Club
Welly Friends
mfer
The Art of Seasons
Looki
Primera
Azuki
Sandbox Land
Imaginary Ones
CyberKongz VX
Cybernetic Harambes
DigiDaigakuSpirits
Gutter Dogs
CNP Jobs
DEGEN TOONZ
VeeFriends Series 2
Porkers
Judy Rabbit
VaynerSports Pass
ATARI50
Wolf Game
Moonbirds Oddities
Keungz Genesis
MetaHero
Hume Genesis
golden egg club
Otherdeed
posers
YOLO Bunny
10KTF
Metamorphic.SevenTreasures
Pota

In [232]:
"""
nft_tweets_demo backfill for the first time
"""
import requests
import pandas as pd
import numpy as np
import sqlalchemy as db
import time

engine = db.create_engine("")
connection = engine.connect()
bearer_token = ''

# collections = pd.read_sql_query('select collection_name from nft.collection_meta_data', onchain_connection)
# collection = collections.sample(1) #how to go through each row 

headers = {'Authorization': f"Bearer {bearer_token}",}
params = {
    'query': 'Primera' + ' "nft" -is:retweet',
    "tweet.fields": "created_at,public_metrics,text",
    "sort_order": "relevancy",
    "max_results": 100,
    "expansions": "author_id",
}
response = requests.get('https://api.twitter.com/2/tweets/search/recent', params=params, headers=headers)
data = response.json()
data = pd.DataFrame(data['data'])
data = pd.concat([data.drop(['public_metrics'], axis=1), data['public_metrics'].apply(pd.Series)], axis=1)
result = data[['created_at', 'author_id', 'text','retweet_count', 'reply_count', 'like_count', 'id']]
result = result.replace('\n',' ', regex=True)
result = result.replace("'"," ", regex=True)

s = ', '.join(["("+ ', '.join( ["'Primera'", "'" + str(result.iloc[i].created_at)[0:19]+ "'",
              "'"+str(result.iloc[i].author_id)+"'", "'"+str(result.iloc[i].text)+"'", "'" + str(result.iloc[i].retweet_count)+ "'", "'"+str(result.iloc[i].reply_count)+"'", "'"+str(result.iloc[i].like_count)+"'", "'"+str(result.iloc[i].id)+"'"+ ")" ]) for i in range(len(result))])
        
query = f"""
INSERT INTO nft_tweets_demo 
VALUES {s}
ON CONFLICT ON CONSTRAINT unique_tweet_id 
DO NOTHING
"""
try:
    pd.read_sql_query(db.text(query), connection)

except Exception as e:
    print(e)

This result object does not return rows. It has been closed automatically.


In [218]:
response.json()

{'errors': [{'parameters': {'query': ['Bored and Dangerous "nft" -is:retweet']},
   'message': 'There were errors processing your request: Ambiguous use of and as a keyword. Use a space to logically join two clauses, or "and" to find occurrences of and in text (at position 7)'}],
 'title': 'Invalid Request',
 'detail': 'One or more parameters to your request was invalid.',
 'type': 'https://api.twitter.com/2/problems/invalid-request'}

In [99]:
result

,created_at,author_id,text,retweet_count,reply_count,like_count,id
0,2022-09-24T08:05:48.000Z,1415701477,"【アランの行うプレゼント企画】 今、話題沸騰中のNFT #Giveaway 💐💐 ""なん...",367,105,370,1573584469889777669
1,2022-09-23T12:44:29.000Z,1534362791502823424,It s a new NFT project! The person in charge w...,6,5,13,1573292218567245824
2,2022-09-20T02:08:05.000Z,711945892752916481,Late night hangout @KUMALEON_ @avenue_nft http...,0,0,3,1572044899708706818
3,2022-09-19T11:34:02.000Z,1542834275577663493,⚡️KUMALEON Trendingランキング24h 1位 おめでとうございます!!🎉 ...,1541,676,1611,1571824935169052673
4,2022-09-22T17:49:31.000Z,1536372257420980226,"🤯 MAD 7 Days, for Weller NFT Wallet Tracker 🤖 ...",448,11,457,1573006591816761346
...,...,...,...,...,...,...,...
95,2022-09-24T03:29:17.000Z,829711480555597824,@DelGlennDavis @KUMALEON_ Welcome sir! https:/...,0,0,1,1573514883941167104
96,2022-09-20T16:00:23.000Z,3814059323,@kumaleon_ @okazz_ @avenue_nft Minted a @okazz...,2,0,2,1572254352928083968
97,2022-09-18T16:25:19.000Z,1560300752027123712,Finally got my KUMA. Love so much😍😍😍 #KUMA #NF...,0,0,1,1571535853192900609
98,2022-09-22T15:57:13.000Z,2899412307,NFTs monitor by @BlessTiger https://t.co/Ktf3m...,0,2,1,1572978333285240832


In [ ]:
# filtering tweets with hashtags only
tweets = data[data['text'].str.contains("#")].reset_index(drop=True)
hashtags_count = pd.DataFrame(tweets.text.str.extractall(r'(\#\w+)')[0].value_counts())
hashtags_count.columns = ['count']
hashtags_count.reset_index(inplace = True)
hashtags_count['percent'] = (hashtags_count['count'] / hashtags_count['count'].sum()) * 100
hashtags_count['timestamp'] = datetime.datetime.now()
hashtags_count.rename(columns={ hashtags_count.columns[0]: "hashtag" }, inplace = True)

#filter by count nubmer
# result = hashtags_count[(hashtags_count['percent'] >= min_percent)]
result = hashtags_count.head(np.minimum(20, hashtags_count.shape[0]))